In [2]:
from Models import ML_Models
exam_models = ML_Models()

In [3]:
import yfinance as yf
import pandas as pd
import inspect
import datetime as dt
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Parameters used for fetching the data from yfinance.
start_date = "2012-01-01"
end_date = dt.date.today()
main_col = "Adj Close"
interval = "1d"
stocks_list = ["EQNR.OL", "DNB.OL", "TEL.OL", "NHY.OL", "AKRBP.OL", "YAR.OL", "MOWI.OL", "CL=F", "OSEBX.OL"]

# Specifying the indicators wanted for further analysis.
indicators = ["MA5", "MA20", "MA50", "MA200", "MIN", "MAX", "LOG_RET", "MOM", "VOLA", "DIFF"]

# Models to utilize for forecasting/prediction. 
# Name of the models are based on the pick_model function in Models.py
models = ["LR", "DTR", "MLP", "XGBoost", "XGBoost_LR", "ADA", "ADA_LR", "ADA_MLP" "GBR", "Bagging", "Bagging_LR","StackedRegressor"]

# Metrics used to evaluate the performance of each model.
# MAE, MSE, RMSE and MAPE are named with "neg_" to be recognized by the cross_validate function from scikit-learn.
metric_names = ["r2", "neg_mean_absolute_error", "neg_mean_squared_error", "neg_root_mean_squared_error", "neg_mean_absolute_percentage_error"]
pretty_metric_names = {"r2":"R^2: ", "neg_mean_absolute_error":"MAE: ", "neg_mean_squared_error":"MSE: ","neg_root_mean_squared_error":"RMSE: ", "neg_mean_absolute_percentage_error":"MAPE: "}

# Collecting data from Yahoo Finance

In [5]:
stock_data = {}
for ticker in stocks_list:
    print(f"Downloading {ticker} data")
    # fetch stock data from yahoo finance
    raw_data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    stock_data[ticker] = raw_data

print("All the data is now downloaded!")

# Save fetched data to csv
#for ticker in stocks_list:
    #stock_data[ticker].to_csv("raw_data/data_"+ticker+".csv")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
All the data is now downloaded!


# Preproccesing the data

In [6]:
def add_indicator_columns(data, indicators):
    # Creating label and shifting the selected main_col value by 1.
    label_name = "Label"
    data[label_name] = data[main_col].shift(periods=1)

    # Checking which of the different indicators that should be added as a column (based on input from indicators list).
    if "MA5" in indicators:
        data["MA5"] = data[label_name].rolling(5).mean()
    if "MA20" in indicators:
        data["MA20"] = data[label_name].rolling(20).mean()
    if "MA50" in indicators:
        data["MA50"] = data[label_name].rolling(50).mean()
    if "MA200" in indicators:
        data["MA200"] = data[label_name].rolling(200).mean()
    if "MIN" in indicators:
        data["MIN"] = data[label_name].rolling(20).min()
    if "MAX" in indicators:
        data["MAX"] = data[label_name].rolling(20).max()
    log_ret = np.log(data[label_name] / data[label_name].shift(1))
    if "LOG_RET" in indicators:
        data["LOG_RET"] = log_ret
    if "MOM" in indicators:
        data["MOM"] = log_ret.rolling(20).mean()
    if "VOLA" in indicators:
        data["VOLA"] = log_ret.rolling(20).std()
    if "DIFF" in indicators:
        data["DIFF"] = data[label_name] - data[label_name].shift(1)

    # remove empty vals.
    data.dropna(axis=0, inplace=True)

In [7]:
def create_X_y_arrays(data, label_name):
        # array that contains the indicators data.
        X = data.loc[:, indicators].to_numpy()
        # array with the target data (based on main_col).
        y = data[label_name].to_numpy()
        return X, y

In [8]:
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

def create_X_y_train_test_split(X, y, current_stock):
    data = stock_data[current_stock]

    for train_index, test_index in tscv.split(data):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

    return X_train, X_test, y_train, y_test

In [9]:
# Adding the specified indicators to the data.
for ticker, data in stock_data.items():
    add_indicator_columns(data=data, indicators=indicators)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Training models

In [12]:
cv_results = {}
trained_models = {}

def train_models(input_models):
        for ticker, data in stock_data.items():

            X, y = create_X_y_arrays(data=data, label_name="Label")

            X_train, X_test, y_train, y_test = create_X_y_train_test_split(X=X, y=y, current_stock=ticker)

            # Evaluating and training selected models.
            for model_i in input_models:
                model = exam_models.pick_model(model=model_i)
                metric_dict = {}
                for metric_name in metric_names:
                    metric_dict[metric_name] = metric_name

                print(model)
                    
                # using method from sci-kit lib to cross-validate
                cross_val_results = cross_validate(
                    model,
                    X,
                    y,
                    cv=tscv,
                    scoring=metric_dict,
                    return_train_score=True,
                    n_jobs=-1,
                    verbose=0  
                )
                model.fit(X_train, y_train)
                cv_results[ticker+"_model_"+model_i] = cross_val_results 
                trained_models["trained_model_"+model_i+"_"+ticker] = model

        return cv_results

In [13]:
cv_stocks_models = train_models(input_models=models)

LinearRegression()
DecisionTreeRegressor()
MLPRegressor()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)
XGBRegressor(base_score=None, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_type

TypeError: estimator should be an estimator implementing 'fit' method, None was passed

# Predicting values based on trained models 

In [ ]:
stock_predictions = {}
def predict_trained_models(input_models):
    for ticker, data in stock_data.items():

        # Creating X and y arrays for train and test sets.
        X, y = create_X_y_arrays(data=data, label_name= "Label")

        X_train, X_test, y_train, y_test = create_X_y_train_test_split(X=X, y=y, current_stock=ticker)

        last_train_index, last_test_index = None, None

        for train_index, test_index in tscv.split(data):
            last_train_index, last_test_index = train_index, test_index

        prediction = data.loc[data.index[last_test_index], [main_col]].copy(deep=True)
        stock_predictions[ticker] = prediction

        for model_i in input_models:
            model = trained_models["trained_model_"+model_i+"_"+ticker]
            y_pred = model.predict(X_test)
            prediction.loc[:, model_i+" Prediction"] = y_pred

    return stock_predictions

In [ ]:
predicted_stock_data = predict_trained_models(input_models=models)

In [ ]:
for ticker in stocks_list:
    print(ticker)
    display(predicted_stock_data[ticker])

In [ ]:
metrics_df_output = {}
def print_metrics(save_data_to_df=True):
    for ticker, data in stock_data.items():
        print("\n--------", ticker, "--------")

        # prepare dataframe if requested
        if save_data_to_df:
            metrics_df = pd.DataFrame(index=list(pretty_metric_names.values()))
            metrics_df_output[ticker] = metrics_df
            current_metrics_df = metrics_df

        for model_name in models:        
            cv = cv_results[ticker+"_model_"+model_name]

            print("-", model_name, "-")
            print("-", "Training Scores:", "-")
            split1_errors_string = f"Score for first data split \n"
            split3_errors_string = f"Score for third data split \n"
            split5_errors_string = f"Score for last data split \n"

            for error_metric_name in metric_names:
                # some metrics are saved as negative, so change sign
                if error_metric_name.startswith("neg"):
                    try:
                        error_metric_value = -cv["train"+"_"+error_metric_name]
                    # string being passed that can't be negative
                    except:
                        error_metric_value = cv["train"+"_"+error_metric_name]
                else:
                    error_metric_value = cv["train"+"_"+error_metric_name]

                if save_data_to_df:
                    # pass string, used to indicate missing metrics
                    if isinstance(error_metric_value, str):
                        current_metrics_df.loc[pretty_metric_names[error_metric_name], model_name+" Model "+"Train"] = error_metric_value
                    # otherwise mean of all splits
                    else:
                        current_metrics_df.loc[pretty_metric_names[error_metric_name], model_name+" Model "+"Train"] = np.mean(error_metric_value)
        
                if isinstance(error_metric_value, str):
                    split1_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split1_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[0]:.3f}\n"

                if isinstance(error_metric_value, str):
                    split3_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split3_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[2]:.3f}\n"
    
                if isinstance(error_metric_value, str):
                    split5_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split5_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[-1]:.3f}\n"

            # print the two strings
            print(split1_errors_string+"\n"+split3_errors_string+"\n"+split5_errors_string+"\n")
            
            print("-", "Testing Scores:", "-")
            split1_errors_string = f"Score for first data split \n"
            split3_errors_string = f"Score for third data split \n"
            split5_errors_string = f"Score for last data split \n"

            for error_metric_name in metric_names:
                # some metrics are saved as negative, so change sign
                if error_metric_name.startswith("neg"):
                    try:
                        error_metric_value = -cv["test"+"_"+error_metric_name]
                    # string being passed that can't be negative
                    except:
                        error_metric_value = cv["test"+"_"+error_metric_name]
                else:
                    error_metric_value = cv["test"+"_"+error_metric_name]

                if save_data_to_df:
                    # pass string, used to indicate missing metrics
                    if isinstance(error_metric_value, str):
                        current_metrics_df.loc[pretty_metric_names[error_metric_name], model_name+" Model "+"Test"] = error_metric_value
                    # otherwise mean of all splits
                    else:
                        current_metrics_df.loc[pretty_metric_names[error_metric_name], model_name+" Model "+"Test"] = np.mean(error_metric_value)
        
                if isinstance(error_metric_value, str):
                    split1_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split1_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[0]:.3f}\n"

                if isinstance(error_metric_value, str):
                    split3_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split3_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[2]:.3f}\n"

                if isinstance(error_metric_value, str):
                    split5_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split5_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[-1]:.3f}\n"

            # print the two strings
            print(split1_errors_string+"\n"+split3_errors_string+"\n"+split5_errors_string+"\n")

    if save_data_to_df:
        return metrics_df_output
    else:
        return None

In [ ]:
metrics_output = print_metrics(save_data_to_df=True)

# Plots for insights about the data

In [ ]:
# PLOT FOR SHOWING DIFFERENT DATA SPLITS FOR THE DIFFERENT STOCKS
for ticker in stocks_list:
    fig, sub_plots = plt.subplots(n_splits, figsize=(16,20))
    fig.set_tight_layout(True)

    data = stock_data[ticker]
    idx = data.index

    tscv = TimeSeriesSplit(n_splits=n_splits)

    splits = list(tscv.split(data))
        
    current_split = 1
    for i in range(len(sub_plots)):
        train_index, test_index = splits[i]

        sub_plots[i].plot(idx[train_index], data.loc[idx[train_index], main_col], label=f"Training data {current_split}", color="blue")
        sub_plots[i].plot(idx[test_index], data.loc[idx[test_index], main_col], label=f"Test data {current_split}", color="red")
        sub_plots[i].set_xlim(idx[0], idx[-1])
        sub_plots[i].set_title(f"Train / test split {current_split} for {ticker}")
        sub_plots[i].set_xlabel("Date")
        sub_plots[i].set_ylabel(f"{main_col}")
        sub_plots[i].legend()

        current_split = current_split + 1
    
    #fig.savefig("data_splits_plots/"+ticker+".png")

fig.show()

In [ ]:
# PLOT FOR COMPARING ACTUAL TO THE DIFFERENT MODELS PREDICTED VALUES
for ticker, data in stock_data.items():
    figure, axs = plt.subplots(figsize=(32,16))
    
    X, y = create_X_y_arrays(data=data, label_name="Label")
    X_train, X_test, y_train, y_test = create_X_y_train_test_split(X=X, y=y, current_stock=ticker)

    X_test_index = np.arange(X_train.shape[0], X_train.shape[0]+X_test.shape[0])

    plt.plot(data.index[X_test_index], y_test, color='purple', label='Actual', linewidth="4.0")
    for model in models:
        plt.plot(data.index[X_test_index], stock_predictions[ticker][model+" Prediction"], label=model)
    plt.title(f'Actual vs Predicted, {ticker}')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()

    #plt.savefig("actual_predicted_plots/"+ticker+".png")
    plt.show()

In [ ]:
# CORRELATION PLOT BETWEEN THE DIFFERENT STOCKS
adj_close_prices = pd.DataFrame({i: j[main_col] for i, j in stock_data.items()})

corr = adj_close_prices.corr()

sns.heatmap(corr, cmap="coolwarm", annot=True)

plt.title("Correlation between stocks")

#plt.savefig("correlation_plots/correlation_stocks.png")
plt.show()

# Saving the data

In [ ]:
for ticker in stocks_list:
    #metrics_output[ticker].to_csv("saved_metrics/stock_"+ticker+".csv")
    #stock_data[ticker].to_pickle("saved_data_pickle/stock_"+ticker+".pkl")
    #predicted_stock_data[ticker].to_csv("saved_predictions/predicted_data_"+ticker+".csv")

In [ ]:
for t, d in stock_data.items():
    print(t)
    print(d["VOLA"])